In [2]:
#Importing packages we need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt #pyplot is used to plot the data
import scipy.optimize as opt #used to numerically optimize
from datetime import time, timedelta, datetime
def GaussianDensity(y,m,s):
    return np.exp(-0.5*np.log(np.pi)-0.5*np.log(s)-0.5*((y-m)**2)/s)

#getting data
#Locate my folder
data=np.genfromtxt('SP_500.csv', delimiter=',',usecols=np.arange(0,4)) #loading in first 4 columns
y = data[15097:, 3:4]*100 # 100 times log-returns of the S&P 500 index. January 4, 2010 - till end
y=y.T[0,:] #unpacking numpy array
T = len(y) #length of time series

def likelihood(gamma):
    #parameters
    p00    = gamma[0]
    p11    = gamma[1]
    sigma2 = gamma[2:4]**2;
    T      = len(y)
    #//transition matrix
    P = np.zeros([2,2])
    P[0]=p00, 1-p11
    P[1]=1-p00, p11

    #//bookkeeping
    xi_10      = np.zeros([2,T+1])
    xi_11      = np.zeros([2,T])
    xi_1T      = np.zeros([2,T])
    lik        = np.zeros(T)

    #//regression:
    A  = np.vstack(((np.identity(2)-P),np.ones([1,2])))
    pi_first = np.linalg.inv(A.T.dot(A)).dot(A.T)
    pi_second=np.vstack((np.zeros([2,1]),np.ones([1,1])))
    pi=pi_first.dot(pi_second)
    xi_10[[0,1],0] = pi.T
    print(pi.T)
    #//forward filter recursion
    eta=np.zeros(2)

    for t in range(T):
        #//state densities
        eta[0]=GaussianDensity(y[t],0,sigma2[0])
        eta[1]=GaussianDensity(y[t],0,sigma2[1])
        
        #likelihood
        #print(np.log(xi_10[[0,1],t]))
        lik[t]   = np.log(xi_10[0,t]*eta[0]+xi_10[1,t]*eta[1])
        
        #filtering
        num0=eta[0]*xi_10[0,t]/(eta[0]*xi_10[0,t]+eta[1]*xi_10[1,t])
        num1=eta[1]*xi_10[1,t]/(eta[0]*xi_10[0,t]+eta[1]*xi_10[1,t])
        xi_11[[0,1],t] = num0,num1

        #prediction
        xi_10[[0,1],t+1] = P.dot(xi_11[[0,1],t])
    #print(GaussianDensity(y[1],0,sigma2[0]))
    return -np.sum(lik) #We wish to minimize the likelihood in our scipy opt function

sigma2 = np.var(y) #used for initial guesses for sigma2 vals
Gamma0  =np.array([0.95,0.95,np.sqrt(2*sigma2),np.sqrt(0.5*sigma2)]) #initial guesses
res=opt.minimize(likelihood, Gamma0, method='L-BFGS-B',bounds=((0.001,0.9999),(0.001,0.9999),(0.01,None),(0.01,None))) #optimizing. We use L-BFGS-B as it allows for bounds and can compute the standard errors (from the inverse hessian) right away
res.x
v_hessian=res.hess_inv.todense() #retrieves the negative inverse hessian matrix (note we have minimized the negative log likelihood function)
se_hessian=np.sqrt(np.diagonal(v_hessian))

FileNotFoundError: SP_500.csv not found.